In [ ]:
pip install scrapy

In [ ]:
import scrapy
import logging
from scrapy.crawler import CrawlerRunner

#custom spider to crawl IAO website
class IAOCrawler(scrapy.Spider):
    name = "IAO Blog-Crawler"
    start_urls = [
        'https://blog.iao.fraunhofer.de/',
    ]
    
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'FEEDS' : {
            'results.json': {
                'format': 'json',
                'overwrite': True,
                'encoding': 'utf-8'
            }
        }
    }
    
    
    #yields data of individual blog-posts
    def parse(self, response):
        containers = response.css('div.entry-container')
        for entry in containers:
            yield {
                'titel': entry.xpath('.//header/h2/span/a/text()').get(),
                'date': entry.css('time.entry-date::text').get(),
                'author': entry.xpath('.//span[@class="author"]/a/text()').get(),
                'author_link': entry.xpath('.//a[@class="entry-author-link"]').attrib['href'],
                'series': entry.xpath('.//div//b/text()').get(default="none"),
                'comments': entry.xpath('//span[@class="comments-link"]/a/@href').get("comments disabled"),
            }
        
        #creates a http request, loading the next page
        next_page = response.xpath('.//div[@id="next-posts"]/a').attrib['href']
        print("scraping:" + next_page)
        yield scrapy.Request(next_page)
    
        

def run():
    crawler = CrawlerRunner()
    return crawler.crawl(IAOCrawler)

#runs the crawler
run()

